In [1]:
import pandas as pd
import zipfile 
import glob, os
import numpy as np
import pylab as pl
import json
import urllib.request
import statsmodels.formula.api as smf
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
nyc2012 = pd.read_csv('nycBuildings2012.csv')
nyc2014 = pd.read_csv('nycBuildings2014.csv')

In [139]:
print(nyc2012.siteEUI.sum())
print(nyc2014.siteEUI.sum())
print(nyc2012.GHG.sum())
print(nyc2014.GHG.sum())

607643.2999999985
666215.500000003
8572288.07
7130779.199999964


In [7]:
print(nyc2012.columns)
print(nyc2014.columns)

Index(['Unnamed: 0', 'siteEUI', 'floorArea', 'ZipCode', 'numBldgs', 'UnitsRes',
       'UnitsTotal', 'YearBuilt', 'LotArea', 'NumFloors', 'ResArea',
       'RetailArea', 'AssessTot', 'CommFAR', 'FacilFAR', 'BuiltFAR',
       'ResidFAR', 'LotArea.1', 'LotFront', 'LotDepth', 'BldgFront', 'GHG',
       'CB2010', 'CT2010', 'water', 'ProxCode', 'LtdHeight', 'HistDist',
       'BldgArea', 'ComArea', 'OfficeArea', 'RetailArea.1', 'GarageArea',
       'StrgeArea', 'FactryArea', 'OtherArea', 'Reported BINs', 'totalEUI',
       'age20', 'age40', 'age60', 'age60ab', 'nonRes', 'income'],
      dtype='object')
Index(['Unnamed: 0', 'siteEUI', 'ZipCode', 'numBldgs', 'UnitsRes',
       'UnitsTotal', 'YearBuilt', 'LotArea', 'NumFloors', 'ResArea',
       'RetailArea', 'AssessTot', 'CommFAR', 'FacilFAR', 'BuiltFAR',
       'ResidFAR', 'LotArea.1', 'LotFront', 'LotDepth', 'BldgFront',
       'DirectGHG', 'InderectGHG', 'floorArea', 'CB2010', 'CT2010', 'GHG',
       'water', 'ProxCode', 'LtdHeight', 'Hist

In [10]:
nyc2012bin = nyc2012.rename(columns={'Reported BINs': 'bins'})
nyc2014bin = nyc2014.rename(columns={'Reported NYC Building Identification Numbers (BINs)': 'bins'})

In [67]:
nyc2012bin.bins = nyc2012bin.bins.astype(str)

In [82]:
nyc2012bin = nyc2012bin[nyc2012bin['bins'].apply(lambda x: str(x).isnumeric())]

In [72]:
nyc2014bin = nyc2014bin[nyc2014bin['bins']  >=0 ]

In [87]:
nyc2012bin.bins = nyc2012bin.bins.astype(float)

In [86]:
nyc2014bin.bins = nyc2014bin.bins.astype(float)

In [92]:
nyc2012binEG = nyc2012bin[['bins', 'GHG', 'siteEUI']]
nyc2014binEG = nyc2014bin[['bins', 'GHG', 'siteEUI']]

In [102]:
print(len(unique(nyc2012binEG.bins)))

5248


In [103]:
print(len(unique(nyc2014binEG.bins)))

5247


In [106]:
nyc2012binEG = nyc2012binEG.drop_duplicates(cols='bins', take_last=True)
print(len(nyc2012binEG.bins))

5248


/Users/MSAUSI2015/anaconda/lib/python3.4/site-packages/pandas/util/decorators.py:81: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  warnings.warn(msg, FutureWarning)


In [131]:
nyc2014binEG = nyc2014binEG.drop_duplicates(cols='bins', take_last=False)
print(len(nyc2014binEG.bins))

5247


/Users/MSAUSI2015/anaconda/lib/python3.4/site-packages/pandas/util/decorators.py:81: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  warnings.warn(msg, FutureWarning)


In [142]:
print(nyc2012binEG.siteEUI.sum()/len(nyc2012binEG))
print(nyc2014binEG.siteEUI.sum()/len(nyc2014binEG))
print(nyc2012binEG.GHG.sum()/len(nyc2012binEG))
print(nyc2014binEG.GHG.sum()/len(nyc2014binEG))

76.2687309451218
86.65469792262238
698.7627362804865
746.2858014103323


In [132]:
print(len(nyc2012binEG))
print(len(nyc2014binEG))
nyc2014and2012 = pd.merge(nyc2012binEG,nyc2014binEG, how='inner',on='bins', suffixes=('_12', '_14'))
len(nyc2014and2012)

5248
5247


4321

In [133]:
nyc2014and2012['decreaseGHG'] = nyc2014and2012.GHG_12 > nyc2014and2012.GHG_14

In [134]:
nyc2014and2012['decreaseSiteEUI'] = nyc2014and2012.siteEUI_12 > nyc2014and2012.siteEUI_14

In [135]:
decreasedGHG = nyc2014and2012.GHG_12.sum() - nyc2014and2012.GHG_14.sum()
print(decreasedGHG)

-115442.03000000818


In [136]:
nyc2014and2012.head()

,bins,GHG_12,siteEUI_12,GHG_14,siteEUI_14,decreaseGHG,decreaseSiteEUI
0,1028241,330.36,62.4,339.1,65.4,False,False
1,3183899,430.36,49.7,510.6,66.9,False,False
2,1031130,315.64,60.9,331.0,68.6,False,False
3,1086174,2633.83,112.4,2456.7,108.7,True,True
4,1086547,1853.76,96.3,1676.8,89.9,True,True


In [137]:
decreasedEUI = nyc2014and2012.siteEUI_12.sum() - nyc2014and2012.siteEUI_14.sum()
print(decreasedEUI/len(nyc2014and2012))

-10.436264753529406
